# Dataset
The data is contained in three files:

portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)

profile.json - demographic data for each customer

transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

# portfolio.json

id (string) - offer id

offer_type (string) - type of offer ie BOGO, discount, informational

difficulty (int) - minimum required spend to complete an offer

reward (int) - reward given for completing an offer

duration (int) - time for offer to be open, in days

channels (list of strings)

# profile.json

age (int) - age of the customer

became_member_on (int) - date when customer created an app account

gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)

id (str) - customer id

income (float) - customer's income

# transcript.json

event (str) - record description (ie transaction, offer received, offer viewed, etc.)

time (int) - time in hours since start of test. The data begins at time t=0

value - (dict of strings) - either an offer id or transaction amount depending on the record


In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
portfolio = pd.read_json('C:/Users/Srujan/Documents/Datasets/Starbucks/portfolio.json',lines=True)
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded_channels = mlb.fit_transform(portfolio['channels'])
encoded_channels

array([[1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 0, 1],
       [1, 1, 0, 1],
       [1, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 0, 1]])

In [4]:
encoded_channels = pd.DataFrame(encoded_channels,columns=mlb.classes_)
encoded_channels

,email,mobile,social,web
0,1,1,1,0
1,1,1,1,1
2,1,1,0,1
3,1,1,0,1
4,1,0,0,1
5,1,1,1,1
6,1,1,1,1
7,1,1,1,0
8,1,1,1,1
9,1,1,0,1


In [5]:
portfolio.drop(['channels'],axis=1,inplace=True)
portfolio = pd.concat([portfolio,encoded_channels],axis=1)
portfolio

,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


In [6]:
profile = pd.read_json('C:/Users/Srujan/Documents/Datasets/Starbucks/profile.json',lines=True)
profile

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [7]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [8]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'],format="%Y%m%d")

In [9]:
profile['year joined'] = profile['became_member_on'].apply(lambda x: str(x.year))
profile

,gender,age,id,became_member_on,income,year joined
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,2017
...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,2018
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,2018
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,2017
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,2016


In [10]:
profile.describe()

,age,income
count,17000.000000,14825.000000
mean,62.531412,65404.991568
std,26.738580,21598.299410
min,18.000000,30000.000000
25%,45.000000,49000.000000
50%,58.000000,64000.000000
75%,73.000000,80000.000000
max,118.000000,120000.000000


Maximum age is 118 which needs to be removed. Age above 100 will be dropped

In [11]:
# imputing null values

profile['income'].fillna(profile['income'].mean(),inplace=True)
profile['gender'].fillna(profile['gender'].mode(),inplace=True)

profile  = profile.loc[profile['age']<100,:]
profile

,gender,age,id,became_member_on,income,year joined
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,2018
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,2018
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,2017
...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,2018
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,2018
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,2017
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,2016


In [12]:
profile['year joined'].value_counts()

2017    5590
2018    3666
2016    3023
2015    1593
2014     662
2013     274
Name: year joined, dtype: int64

In [13]:
profile['member_since(in years)'] = profile['year joined'].apply(lambda x: 2020- int(x))

C:\Users\Srujan\Anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# gender type dummies

gender_dummies = profile['gender'].str.get_dummies().add_prefix('gender_')
#year_joined_dummies = profile['year joined'].str.get_dummies().add_prefix('year_joined_')

profile = pd.concat([profile, gender_dummies], axis=1)

profile.drop(['became_member_on'], axis=1, inplace=True)

profile.head()

,gender,age,id,income,year joined,member_since(in years),gender_F,gender_M,gender_O
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017,3,1,0,0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017,3,1,0,0
5,M,68,e2127556f4f64592b11af22de27a7932,70000.0,2018,2,0,1,0
8,M,65,389bc3fa690240e798340f5a15918d5c,53000.0,2018,2,0,1,0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017,3,0,1,0


In [15]:
profile.describe()

,age,income,member_since(in years),gender_F,gender_M,gender_O
count,14808.000000,14808.000000,14808.000000,14808.000000,14808.000000,14808.000000
mean,54.340829,65394.313884,3.379862,0.412952,0.572798,0.014249
std,17.323921,21595.072904,1.198429,0.492381,0.494689,0.118520
min,18.000000,30000.000000,2.000000,0.000000,0.000000,0.000000
25%,42.000000,49000.000000,3.000000,0.000000,0.000000,0.000000
50%,55.000000,64000.000000,3.000000,0.000000,1.000000,0.000000
75%,66.000000,80000.000000,4.000000,1.000000,1.000000,0.000000
max,99.000000,120000.000000,7.000000,1.000000,1.000000,1.000000


In [16]:
#binning  the income into 4 quantiles as we can observe if income 
bin_labels_5 = [1, 2, 3, 4]
profile['income bins'] = pd.qcut(profile['income'],
                              q=[0, .25, .50, .75, 1],
                              labels=bin_labels_5)
profile

,gender,age,id,income,year joined,member_since(in years),gender_F,gender_M,gender_O,income bins
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017,3,1,0,0,4
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017,3,1,0,0,4
5,M,68,e2127556f4f64592b11af22de27a7932,70000.0,2018,2,0,1,0,3
8,M,65,389bc3fa690240e798340f5a15918d5c,53000.0,2018,2,0,1,0,2
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017,3,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,2018,2,1,0,0,2
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,2018,2,0,1,0,3
16997,M,49,01d26f638c274aa0b965d24cefe3183f,73000.0,2017,3,0,1,0,3
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,2016,4,1,0,0,2


In [17]:
transcript = pd.read_json('C:/Users/Srujan/Documents/Datasets/Starbucks/transcript.json',lines=True)
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [18]:
transcript['time'].value_counts()

408    17030
576    17015
504    16822
336    16302
168    16150
       ...  
318      940
330      938
156      914
162      910
150      894
Name: time, Length: 120, dtype: int64

In [19]:
transactions = transcript.loc[transcript['event']=='transaction',:]
transactions['amount'] = transactions['value'].apply(lambda x: list(x.values())[0])
transactions.drop(['value'],axis=1,inplace=True)
transactions

C:\Users\Srujan\Anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Srujan\Anaconda\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,person,event,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53


In [20]:
offers = transcript.loc[transcript['event']!='transaction',:]
offers['offer_id'] = offers['value'].apply(lambda x: list(x.values())[0])
offers.drop(['value'],axis=1,inplace=True)
offers

C:\Users\Srujan\Anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Srujan\Anaconda\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,person,event,time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,2298d6c36e964ae4a3e7e9706d1fb8c2
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4


# Merging Dataframes

In [21]:
offers = offers.merge(profile,left_on='person',right_on='id',how='inner')
offers = offers.merge(portfolio,left_on='offer_id',right_on='id',how='inner')
offers.drop(['id_x','id_y'],inplace=True,axis=1)
offers

,person,event,time,offer_id,gender,age,income,year joined,member_since(in years),gender_F,...,gender_O,income bins,reward,difficulty,duration,offer_type,email,mobile,social,web
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,0,4,5,5,7,bogo,1,1,0,1
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,0,4,5,5,7,bogo,1,1,0,1
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,0,4,5,5,7,bogo,1,1,0,1
3,e2127556f4f64592b11af22de27a7932,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,0,3,5,5,7,bogo,1,1,0,1
4,e2127556f4f64592b11af22de27a7932,offer viewed,420,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,0,3,5,5,7,bogo,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148623,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,576,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,94000.0,2017,3,0,...,0,4,10,10,5,bogo,1,1,1,1
148624,3f3494f3234e4206a279662bef81fe85,offer received,576,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,0,2,10,10,5,bogo,1,1,1,1
148625,3f3494f3234e4206a279662bef81fe85,offer viewed,594,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,0,2,10,10,5,bogo,1,1,1,1
148626,3045af4e98794a04a5542d3eac939b1f,offer received,576,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,78000.0,2016,4,1,...,0,3,10,10,5,bogo,1,1,1,1


In [22]:
event_dummies = pd.get_dummies(offers['event'])
offers = pd.concat([offers,event_dummies],axis=1)
offers

,person,event,time,offer_id,gender,age,income,year joined,member_since(in years),gender_F,...,difficulty,duration,offer_type,email,mobile,social,web,offer completed,offer received,offer viewed
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,5,7,bogo,1,1,0,1,0,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,5,7,bogo,1,1,0,1,0,0,1
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,5,7,bogo,1,1,0,1,1,0,0
3,e2127556f4f64592b11af22de27a7932,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,5,7,bogo,1,1,0,1,0,1,0
4,e2127556f4f64592b11af22de27a7932,offer viewed,420,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,5,7,bogo,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148623,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,576,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,94000.0,2017,3,0,...,10,5,bogo,1,1,1,1,0,0,1
148624,3f3494f3234e4206a279662bef81fe85,offer received,576,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,10,5,bogo,1,1,1,1,0,1,0
148625,3f3494f3234e4206a279662bef81fe85,offer viewed,594,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,10,5,bogo,1,1,1,1,0,0,1
148626,3045af4e98794a04a5542d3eac939b1f,offer received,576,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,78000.0,2016,4,1,...,10,5,bogo,1,1,1,1,0,1,0


In [23]:
offers['time'] = offers['time']+0.5
offers['received_time'] = offers['offer received']*offers['time']
offers['viewed_time'] = offers['offer viewed']*offers['time']
offers['completed_time'] = offers['offer completed']*offers['time']
offers

,person,event,time,offer_id,gender,age,income,year joined,member_since(in years),gender_F,...,email,mobile,social,web,offer completed,offer received,offer viewed,received_time,viewed_time,completed_time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0.5,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,1,1,0,1,0,1,0,0.5,0.0,0.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,6.5,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,1,1,0,1,0,0,1,0.0,6.5,0.0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,132.5,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,100000.0,2017,3,1,...,1,1,0,1,1,0,0,0.0,0.0,132.5
3,e2127556f4f64592b11af22de27a7932,offer received,408.5,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,1,1,0,1,0,1,0,408.5,0.0,0.0
4,e2127556f4f64592b11af22de27a7932,offer viewed,420.5,9b98b8c7a33c4b65b9aebfe6a799e6d9,M,68,70000.0,2018,2,0,...,1,1,0,1,0,0,1,0.0,420.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148623,9fcbff4f8d7241faa4ab8a9d19c8a812,offer viewed,576.5,4d5c57ea9a6940dd891ad53e9dbe8da0,M,47,94000.0,2017,3,0,...,1,1,1,1,0,0,1,0.0,576.5,0.0
148624,3f3494f3234e4206a279662bef81fe85,offer received,576.5,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,1,1,1,1,0,1,0,576.5,0.0,0.0
148625,3f3494f3234e4206a279662bef81fe85,offer viewed,594.5,4d5c57ea9a6940dd891ad53e9dbe8da0,F,61,60000.0,2014,6,1,...,1,1,1,1,0,0,1,0.0,594.5,0.0
148626,3045af4e98794a04a5542d3eac939b1f,offer received,576.5,4d5c57ea9a6940dd891ad53e9dbe8da0,F,58,78000.0,2016,4,1,...,1,1,1,1,0,1,0,576.5,0.0,0.0


In [ ]:
offers[offers.duplicated(subset=['person','offer_id'],keep=False)].head()

In [ ]:
transactions = transactions.merge(profile,left_on='person',right_on='id',how='inner')
transactions.drop('id',inplace=True,axis=1)
transactions

In [ ]:
num_transactions = transactions['person'].value_counts().to_dict()
num_transactions

Same person has availed offers many times. It may be a single offer or multiple offers which that customer has got.

So creating a new column of number of transations of the customer.

In [ ]:
transactions['num_transactions'] = transactions['person']
transactions['num_transactions'] = transactions['num_transactions'].replace(num_transactions)
transactions

In [ ]:
'''from joblib import Parallel,delayed

transactions['num_transactions'] = transactions['person']

def replace(i):
    
    num_trans = num_transactions[i]
    return num_trans


num_trans = Parallel(n_jobs=-1)(delayed(replace)(i) for i in transactions['num_transactions'])
transactions['num_transactions'] = num_trans
transactions'''

In [ ]:
transactions.to_csv('transactions.csv')

In [25]:
offers.to_csv('offers.csv')